In [47]:
import pandas as pd
import json
import re
import os

gpt35 = pd.read_csv("NEW_data_collection/data/gpt35_qa.csv")
gpt4 = pd.read_csv("NEW_data_collection/data/gpt4_qa.csv")

def get_questions(raw_text_col):
    api_outputs = [json.loads(api_output)['choices'][0]['message']['content'] for api_output in raw_text_col]
    json_output = [json.loads(api_output.strip()[7:-3]) for api_output in api_outputs]

    return json_output

gpt35['cleaned_qa'] = get_questions(gpt35['questions'])
gpt4['cleaned_qa'] = get_questions(gpt4['questions'])

# Get average costs associated with each model too
avg_prompt_tokens = sum([json.loads(gpt35.iloc[idx, 2])['usage']['prompt_tokens'] for idx in range(len(gpt35))])/len(gpt35)
avg_completion_tokens = sum([json.loads(gpt35.iloc[idx, 2])['usage']['completion_tokens'] for idx in range(len(gpt35))])/len(gpt35)

avg_cost_per_page = 0.001 * (avg_prompt_tokens/1000) + 0.002 * (avg_completion_tokens/1000)
print(f"GPT-3.5 cost per 1000: ${round(avg_cost_per_page*1000, 3)}")

avg_prompt_tokens = sum([json.loads(gpt4.iloc[idx, 2])['usage']['prompt_tokens'] for idx in range(len(gpt4))])/len(gpt4)
avg_completion_tokens = sum([json.loads(gpt4.iloc[idx, 2])['usage']['completion_tokens'] for idx in range(len(gpt4))])/len(gpt4)

avg_cost_per_page = 0.01 * (avg_prompt_tokens/1000) + 0.03 * (avg_completion_tokens/1000)
print(f"GPT-4 cost per 1000: ${round(avg_cost_per_page*1000, 4)}")

GPT-3.5 cost per 1000: $1.249
GPT-4 cost per 1000: $17.0996


In [58]:
idx = 6
print("GPT-3.5 URL: "+gpt35['url'][idx])
print("GPT-4 URL: "+gpt4['url'][idx])
print("Context:\n"+gpt35['data'][idx])
print("GPT-3.5:\n"+json.dumps(gpt35['cleaned_qa'][idx], indent=4))
print("GPT-4:\n"+json.dumps(gpt4['cleaned_qa'][idx], indent=4))

GPT-3.5 URL: https://www2.brockport.edu/academics/eagle-success/
GPT-4 URL: https://www2.brockport.edu/academics/eagle-success/
Context:
Login to EagleSUCCESS
We believe a campus community engaged with our students will lead to their success. EagleSUCCESS is a collaborative tool for instructors, advisors, and students. With it we can more quickly identify students who are struggling and implement interventions with students in a timely way. How does using EagleSUCCESS increase student success?
Students can:
- Advocate for themselves and ask for help.
- Know their progress.
- Schedule advisement and tutoring appointments.
- See all campus resources listed in one place.
Advisors can:
- See a fuller picture of each student and their performance.
- Implement interventions early and effectively.
- Connect students to campus resources.
Instructors can:
- Share concerns with students and advisors.
- Refer students to Tutoring and other resources.
- Celebrate student achievements.
Learn How to